## change to code and run on colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.10
!pip install wandb
import tensorflow as tf
print(tf.__version__)

In [1]:
import warnings 
warnings.filterwarnings(action='ignore')
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import inception_resnet_v2, nasnet
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback
import wandb
import tensorflow as tf
print(tf.__version__)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.10.0
Num GPUs Available:  1


# Activate this cell when you need
# split data
ori_flood = '../data/binary/flood'
ori_normal = '../data/binary/normal'

img_flood = glob(ori_flood+'/*.jpg')
img_normal = glob(ori_normal+'/*.jpg')

len_flood=len(img_flood)
print('len_flood : ', len_flood)
len_normal=len(img_normal)
print('len_normal : ', len_normal)

train_dir = '../data/binary/train'
valid_dir = '../data/binary/valid'

# copy flood img
for i, img in enumerate(tqdm(img_flood)):
    img_name = img.split('/')[-1]
    src = img
    if i < len_flood * 0.7 :
        dst = os.path.join(train_dir+'/flood', img_name)
    else :
        dst = os.path.join(valid_dir+'/flood', img_name)
    shutil.copyfile(src,dst)

# copy normal img
for i, img in enumerate(tqdm(img_normal)):
    img_name = img.split('/')[-1]
    src = img
    if i < len_normal * 0.7 :
        dst = os.path.join(train_dir+'/normal', img_name)
    else :
        dst = os.path.join(valid_dir+'/normal', img_name)
    shutil.copyfile(src,dst)

# Make DataFrame

In [2]:
# flood : 1, normal : 0
flood_df = pd.DataFrame({'path' : glob('../data/binary/flood/*.jpg'),
                        'label' : '1'})
print('flood len : {}'.format(len(flood_df)))
normal_df = pd.DataFrame({'path' : glob('../data/binary/normal/*.jpg'),
                         'label': '0'})
print('normal len : {}'.format(len(normal_df)))

df = pd.concat([flood_df, normal_df], axis=0)

#shuffle
df = df.sample(frac=1)
display(df.head())

#split
x_train, x_test, y_train, y_test = train_test_split(df['path'],
                                                   df['label'],
                                                   stratify=df['label'],
                                                   test_size=0.2)
train_df = pd.DataFrame({'path':x_train,
                        'label':y_train})
valid_df = pd.DataFrame({'path':x_test,
                        'label':y_test})
train_df.head()

flood len : 6866
normal len : 7723


,path,label
5883,../data/binary/flood/26538299.jpg,1
2929,../data/binary/flood/26544507.jpg,1
2878,../data/binary/flood/26491104.jpg,1
5682,../data/binary/flood/59072394.jpg,1
4987,../data/binary/flood/29_jpg.rf.9c91984bc77fc96...,1


,path,label
678,../data/binary/flood/26533851.jpg,1
7721,../data/binary/normal/frame15_26_day_1024.mp4.jpg,0
6646,../data/binary/flood/26520189.jpg,1
5244,../data/binary/normal/frame37_135_night_1024.m...,0
4537,../data/binary/normal/도로561.jpg,0


# Config

In [3]:
# train_dir = '/content/drive/MyDrive/융합캡스톤디자인/binary/train'
# valid_dir = '/content/drive/MyDrive/융합캡스톤디자인/binary/valid'
# train_dir = '../data/binary/train'
# valid_dir = '../data/binary/valid'

CLASSES = ['normal', 'flood']
NasNet = 'NasNet'
Inception = 'Inception'
target_size = {NasNet : (331,331),
               Inception : (299,299)}
EPOCHS = 1000

sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'early_terminate' : {
        'type': 'hyperband',
        'max_iter': 27,
        's': 2
    },
    'parameters': {
        'pretrain_net': {
            'values': [Inception, NasNet]
        },
        'batch_size': {
            'values': [20, 50]
        },
        'dropout': {
            'values': [0.2, 0.5]
        },
        'dense': {
            'values': [32, 256, 1024]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-5]
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu', 'softmax']
        }
    }
}

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                              rotation_range=30,
                              shear_range=0.2,
                              zoom_range=0.3,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode = 'nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
def train() :
    tf.keras.backend.clear_session()
    config_defaults = {
        'pretrain_net': Inception,
        'epochs' : EPOCHS,
        'batch_size': 50,
        'dropout' : 0.5,
        'learning_rate' : 1e-3,
        'activation': 'relu',
        'optimizer': 'adam',
        'dense':1024
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_generator = train_datagen.flow_from_dataframe(train_df, 
                                               x_col='path',
                                               y_col='label',
                                               target_size=target_size[config.pretrain_net],
                                               class_mode='binary',
                                               batch_size=config.batch_size)
    valid_generator = valid_datagen.flow_from_dataframe(valid_df,
                                               x_col='path',
                                               y_col='label',
                                               target_size=target_size[config.pretrain_net],
                                               class_mode='binary',
                                               batch_size=config.batch_size)    
    
    if config.pretrain_net == Inception :
        base_model = inception_resnet_v2.InceptionResNetV2(
            weights='imagenet',
            include_top = False,
            input_shape = target_size[config.pretrain_net]+(3,)
        )
    elif config.pretrain_net == NasNet :
        base_model= nasnet.NASNetLarge(
              weights='imagenet',
              include_top=False,
              input_shape = target_size[config.pretrain_net]+(3,)
             )
    base_model.trainable = False 

    inputs = layers.Input(shape=target_size[config.pretrain_net]+(3,))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(config.dropout)(x)
    x = layers.Dense(config.dense, activation=config.activation)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    es = EarlyStopping(monitor='loss',
                    mode='auto',
                    patience=5,
                    verbose=1)
    
    ckpt_path = '../data/weight/{net}_{bs}_{do}_{lr}_{acti}_{opt}_{dense}.ckpt'.format(net=config.pretrain_net, 
                                                                                       bs=config.batch_size, 
                                                                                       do=config.dropout, 
                                                                                       lr=config.learning_rate,
                                                                                       acti=config.activation,
                                                                                       opt=config.optimizer,
                                                                                       dense=config.dense)
    checkpointer = ModelCheckpoint(filepath=ckpt_path,
                                  monitor='val_loss',
                                  save_weights_only = True,
                                  save_best_only= True,
                                  verbose=1)

        
    model = Model(inputs=inputs, 
                  outputs=outputs,
                  name='binary_flood')    
    
    if config.optimizer=='sgd':
        optimizer = SGD(learning_rate=config.learning_rate)
    elif config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate)
    elif config.optimizer=='adam':
        optimizer = Adam(learning_rate=config.learning_rate)


    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])    
    
    _ = model.fit(train_generator,
          validation_data=valid_generator,
          verbose=1,
          epochs=config.epochs,
          callbacks=[WandbCallback(), es, checkpointer],
          steps_per_epoch=len(train_df)//config.batch_size)

In [6]:
sweep_id = wandb.sweep(sweep_config, project='flood24')

Create sweep with ID: q6tqn2u4
Sweep URL: https://wandb.ai/hkleee/flood24/sweeps/q6tqn2u4


In [ ]:
# vavoacy6
wandb.agent('vavoacy6', train) 

wandb: Agent Starting Run: 445vi2bj with config:
wandb: 	activation: relu
wandb: 	batch_size: 20
wandb: 	dense: 32
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	pretrain_net: Inception
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 11671 validated image filenames belonging to 2 classes.
Found 2918 validated image filenames belonging to 2 classes.
Metal device set to: Apple M1


2022-11-03 00:29:22.511797: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-03 00:29:22.512153: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/1000


2022-11-03 00:29:28.175637: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-03 00:29:32.488708: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

INFO:tensorflow:Assets written to: /Users/hkleeeee/Desktop/Flooding24/2022-2-SCS4031-9to6/Model/image_classification/wandb/run-20221103_180814-e9mn2kgj/files/model-best/assets
wandb: Adding directory to artifact (/Users/hkleeeee/Desktop/Flooding24/2022-2-SCS4031-9to6/Model/image_classification/wandb/run-20221103_180814-e9mn2kgj/files/model-best)... Done. 0.9s



Epoch 38: val_loss improved from 0.10125 to 0.09882, saving model to ../data/weight/Inception_50_0.2_0.01_softmax_sgd_256.ckpt
233/233 [==============================] - 1079s 5s/step - loss: 0.1193 - accuracy: 0.9686 - val_loss: 0.0988 - val_accuracy: 0.9774
Epoch 39/1000
233/233 [==============================] - ETA: 0s - loss: 0.1150 - accuracy: 0.9695

INFO:tensorflow:Assets written to: /Users/hkleeeee/Desktop/Flooding24/2022-2-SCS4031-9to6/Model/image_classification/wandb/run-20221103_180814-e9mn2kgj/files/model-best/assets


INFO:tensorflow:Assets written to: /Users/hkleeeee/Desktop/Flooding24/2022-2-SCS4031-9to6/Model/image_classification/wandb/run-20221103_180814-e9mn2kgj/files/model-best/assets
wandb: Adding directory to artifact (/Users/hkleeeee/Desktop/Flooding24/2022-2-SCS4031-9to6/Model/image_classification/wandb/run-20221103_180814-e9mn2kgj/files/model-best)... Done. 0.9s



Epoch 39: val_loss improved from 0.09882 to 0.09626, saving model to ../data/weight/Inception_50_0.2_0.01_softmax_sgd_256.ckpt
233/233 [==============================] - 963s 4s/step - loss: 0.1150 - accuracy: 0.9695 - val_loss: 0.0963 - val_accuracy: 0.9774
Epoch 40/1000
233/233 [==============================] - ETA: 0s - loss: 0.1156 - accuracy: 0.9700
Epoch 40: val_loss did not improve from 0.09626
233/233 [==============================] - 897s 4s/step - loss: 0.1156 - accuracy: 0.9700 - val_loss: 0.1003 - val_accuracy: 0.9774
Epoch 41/1000
233/233 [==============================] - ETA: 0s - loss: 0.1120 - accuracy: 0.9713

In [ ]:
# TODO 
# visualize result of inference of validate images